In [5]:
VAL = 0.1à
import 

In [7]:
import os
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

SEED = 1234
tf.random.set_seed(SEED)

cwd = os.getcwd()

import json
import shutil
import random

# Defining the datasets directory
dataset_dir = os.path.join(cwd, 'MaskDataset')
training_dir = os.path.join(dataset_dir, 'training')
validation_dir = os.path.join(dataset_dir, 'validation')
test_dir = os.path.join(dataset_dir, 'test')

# Create validation directory if it doesn't exist
if not os.path.exists(validation_dir):
    os.makedirs(validation_dir)

# Loading the classes of each image into the memory
train_classes_json_file_name = 'train_gt.json'
train_classes_json_directory = os.path.join(dataset_dir, train_classes_json_file_name)

data = {}

with open(train_classes_json_directory) as json_file:
    data = json.load(json_file)


# Creating folder for each class of image for training and validation datasets
classes = set(data.values())
print(classes)

for class_label in classes:
    class_training_dir = os.path.join(training_dir, str(class_label))
    class_validation_dir = os.path.join(validation_dir, str(class_label))
    if not os.path.exists(class_training_dir):
        os.makedirs(class_training_dir)
    if not os.path.exists(class_validation_dir):
        os.makedirs(class_validation_dir)

# Assigning images to each training folder/class, avoiding to have the same image two times in the same folder
for entry in os.scandir(training_dir):
    if(entry.is_file()):
        file_destination = os.path.join(training_dir, str(data[entry.name]), entry.name)
        if not os.path.isfile(file_destination):
            shutil.copy(entry.path, file_destination)
    
# Choosing random images to be into the validation folders, being able to repeat without cloning images
validation_rate = VAL

for class_label in classes:
    class_training_dir = os.path.join(training_dir, str(class_label))
    class_validation_dir = os.path.join(validation_dir, str(class_label))
    
    for old_entry in os.scandir(class_validation_dir):
        os.remove(old_entry.path)
    
    training_entries = list(os.scandir(class_training_dir))
    validation_size = round(len(training_entries)*validation_rate)
    
    for validation_entry in random.sample(training_entries, validation_size):
        destination = os.path.join(class_validation_dir, validation_entry.name)
        os.rename(validation_entry.path, destination)

ModuleNotFoundError: No module named 'tensorflow'

# Data augmentation

In [ ]:
apply_data_augmentation = True

if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=10,
        height_shift_range=10,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='constant',
        cval=0,
        rescale=1/255.
    )
else:
    train_data_gen = ImageDataGenerator(rescale=1/255.)

valid_data_gen = ImageDataGenerator(rescale=1/255.)
# test_data_gen = ImageDataGenerator(rescale=1/255.)

bs = 8

train_gen = train_data_gen.flow_from_directory(
    training_dir,
    batch_size=bs,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

valid_gen = valid_data_gen.flow_from_directory(
    validation_dir,
    batch_size=bs,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

# test_gen = test_data_gen.flow_from_directory(
#     test_dir,
#     batch_size=bs,
#     class_mode='categorical',
#     shuffle=True,
#     seed=SEED
# )

img_h = 256
img_w = 256

num_classes = len(classes)

train_dataset = tf.data.Dataset.from_generator(
    lambda: train_gen,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 256, 256, 3], [None, num_classes])
)

train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(
    lambda: valid_gen,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 256, 256, 3], [None, num_classes])
)

valid_dataset = valid_dataset.repeat()

# test_dataset = tf.data.Dataset.from_generator(
#     lambda: test_gen,
#     output_types=(tf.float32, tf.float32),
#     output_shapes=([None, 256, 256, 3], [None, num_classes])
# )

# test_dataset = test_dataset.repeat()

# Building the Network

In [15]:
start_f = 8
depth = 5

conv_layer_1 = tf.keras.layers.Conv2D(
            filters=start_f,
            kernel_size=(4, 4),
            strides=(1, 1),
            padding=2,
            input_shape=input_shape
    ),

conv_layer_2 = tf.keras.layers.Conv2D(
            filter=start_f,
            kernel_size=(4,4),
            strides=(1,1),
            padding=2,
            input_shape=conv_layer_1
    )   

NameError: name 'tf' is not defined

In [14]:


model = tf.keras.Sequential()
for i in range(depth):
    if i == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape = [None]
    

    # Convolutional part
    model.add(
        conv_layer_1
        conv_layer_2
    )
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    start_f *= 2

# Fully connected part
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

SyntaxError: invalid syntax (<ipython-input-14-4de52517d9d2>, line 31)